In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Libraries

In [60]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.1 MB/s eta 0:00:00


In [65]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score,precision_recall_fscore_support
from catboost import CatBoostClassifier
import xgboost as xgb
import lightgbm as lgb

#Load the data

In [45]:
data = pd.read_csv("/content/drive/MyDrive/creditcard.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [46]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


#Defining x and y

In [47]:
X = data.drop('Class', axis=1)
y = data['Class']

In [48]:
print("Shape of y:", y.shape)
print("Data type of y:", y.dtype)

Shape of y: (284807,)
Data type of y: int64


In [49]:
ros = RandomOverSampler(random_state=42)
X_train_balanced, y_train_balanced = ros.fit_resample(X, y)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [51]:
# check the class distribution of the resampled dataset
print(pd.Series(y_train_balanced).value_counts())

0    284315
1    284315
Name: Class, dtype: int64


In [52]:
oversampler = RandomOverSampler(random_state=42)
X_train_balanced, y_train_balanced = oversampler.fit_resample(X_train, y_train)

#Random forest

In [31]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [33]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test,y_pred))
print("Confuion Matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.9997191109862715
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     28420
           1       1.00      0.87      0.93        61

    accuracy                           1.00     28481
   macro avg       1.00      0.93      0.96     28481
weighted avg       1.00      1.00      1.00     28481

Confuion Matrix:
 [[28420     0]
 [    8    53]]


#Adaboost Classifier

In [56]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [57]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test,y_pred))
print("Confuion Matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.9991924440855307
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     28439
           1       0.81      0.60      0.68        42

    accuracy                           1.00     28481
   macro avg       0.90      0.80      0.84     28481
weighted avg       1.00      1.00      1.00     28481

Confuion Matrix:
 [[28433     6]
 [   17    25]]


#Catboost Classifier

In [62]:
model = CatBoostClassifier()
model.fit(X_train, y_train, verbose=100)

Learning rate set to 0.110024
0:	learn: 0.3361099	total: 303ms	remaining: 5m 2s
100:	learn: 0.0014509	total: 25s	remaining: 3m 42s
200:	learn: 0.0010515	total: 39.1s	remaining: 2m 35s
300:	learn: 0.0007979	total: 48.3s	remaining: 1m 52s
400:	learn: 0.0005926	total: 56.6s	remaining: 1m 24s
500:	learn: 0.0003899	total: 1m 12s	remaining: 1m 12s
600:	learn: 0.0002587	total: 1m 22s	remaining: 54.9s
700:	learn: 0.0001804	total: 1m 34s	remaining: 40.1s
800:	learn: 0.0001371	total: 1m 47s	remaining: 26.6s
900:	learn: 0.0001081	total: 2m 1s	remaining: 13.4s
999:	learn: 0.0000895	total: 2m 8s	remaining: 0us


In [63]:
print('Training accuracy:', model.score(X_train, y_train))
print('Testing accuracy:', model.score(X_test, y_test))

Training accuracy: 1.0
Testing accuracy: 0.9995786664794073


In [64]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test,y_pred))
print("Confuion Matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.9991924440855307
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     28439
           1       0.81      0.60      0.68        42

    accuracy                           1.00     28481
   macro avg       0.90      0.80      0.84     28481
weighted avg       1.00      1.00      1.00     28481

Confuion Matrix:
 [[28433     6]
 [   17    25]]


#Xgboost

In [66]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [67]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test,y_pred))
print("Confuion Matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.9991924440855307
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     28439
           1       0.81      0.60      0.68        42

    accuracy                           1.00     28481
   macro avg       0.90      0.80      0.84     28481
weighted avg       1.00      1.00      1.00     28481

Confuion Matrix:
 [[28433     6]
 [   17    25]]


#LightGBM

In [68]:
train_data = lgb.Dataset(X_train, label=y_train)

In [69]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [70]:
model = lgb.train(params, train_data, num_boost_round=100)
y_pred = model.predict(X_test)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred]

[LightGBM] [Info] Number of positive: 450, number of negative: 255876
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.097860 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 256326, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001756 -> initscore=-6.343201
[LightGBM] [Info] Start training from score -6.343201


In [71]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test,y_pred))
print("Confuion Matrix:\n", confusion_matrix(y_test,y_pred))

Accuracy: 0.9968048874688389
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     28439
           1       0.23      0.50      0.32        42

    accuracy                           1.00     28481
   macro avg       0.62      0.75      0.66     28481
weighted avg       1.00      1.00      1.00     28481

Confuion Matrix:
 [[28369    70]
 [   21    21]]
